In [ ]:
from IPython.display import clear_output

In [ ]:
%pip install torch torchvision

clear_output()

## Contents

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torchvision.models import resnet18
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10

## Download and Load the dataset

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
])

transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

train_dataset = CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=4)

100%|██████████| 170498071/170498071 [00:01<00:00, 88346597.85it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


## Download and Load the model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# To see how much improvement it is using a pre-trained model and fine tuning it vs training from scratch
# try setting pretrained=False to use the same architecture of resnet 18 but without weights.
resnet18_model = resnet18(pretrained=True)  # download pre-trained model

# Change fc layer to accomodate for 10 classes of Cifar-10
num_ftrs = resnet18_model.fc.in_features
resnet18_model.fc = nn.Linear(num_ftrs, 10)

resnet18_model = resnet18_model.to(device)

In [ ]:
resnet18_model.eval()

correct = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = resnet18_model(inputs)
        predicted = torch.argmax(outputs.data, 1)

        correct += (predicted == labels).sum().item()

print(f"Pre training accuracy on the test set: {100 * correct / len(test_loader.dataset)}%")

Pre fine tuning on the test set: 11.94%


Fine tuning on Cifar-10

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18_model.parameters(), lr=0.001, momentum=0.9)

train_losses = []

num_epochs = 10

for epoch in range(num_epochs):

    resnet18_model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = resnet18_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    train_losses.append(running_loss)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

Epoch 1/10, Loss: 1.3939468054747095
Epoch 2/10, Loss: 0.8340907604718695
Epoch 3/10, Loss: 0.7075368762016296
Epoch 4/10, Loss: 0.6344962740431026
Epoch 5/10, Loss: 0.5867826491594315
Epoch 6/10, Loss: 0.5582046063274754
Epoch 7/10, Loss: 0.5327528780516313
Epoch 8/10, Loss: 0.517860226941352
Epoch 9/10, Loss: 0.4988023795339526
Epoch 10/10, Loss: 0.4794570666794874


Post training accuracy

In [ ]:
resnet18_model.eval()

correct = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        outputs = resnet18_model(inputs)
        _, predicted = torch.max(outputs.data, 1)

        correct += (predicted == labels).sum().item()

print(f"Post training on the test set: {100 * correct / len(test_loader.dataset)}%")

Post training on the test set: 92.68%
